In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

In [0]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 50 # 訓練的 epochs 數量
LEARNING_RATE = 1e-3
optimizers = [RMSprop, Adam]

In [27]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [18]:
x_train.shape[1:]

(32, 32, 3)

In [0]:
def cifar_generator(x, y, batch_size=128):
    while True:
        for indexs in range(0, len(x), batch_size):
            images = x[indexs: indexs+batch_size]
            labels = y[indexs: indexs+batch_size]
            yield images, labels

In [0]:
cifar_gen_train = cifar_generator(x_train, y_train)
cifar_gen_test = cifar_generator(x_test, y_test)

In [0]:
def build_cnn(input_shape, output_units=10, num_neurons=512):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(num_neurons))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(output_units))
  model.add(Activation('softmax'))

  return model

In [35]:
model = build_cnn(input_shape=x_train.shape[1:])
model.summary()

optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit_generator(cifar_gen_train,                      
                    steps_per_epoch=400, # train size 50000 除以 batch size 128 約 400
                    epochs=epochs,
                    verbose=1,
                    validation_steps=80, # 10000 / 128 約 80
                    validation_data=cifar_gen_test)

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

# exp_name_tag = "batch size:{}".format(_batch)  
score = model.evaluate(x_test, y_test, verbose=0)
# results[exp_name_tag] = {'train-loss': train_loss,
#                           'valid-loss': valid_loss,
#                           'train-acc': train_acc,
#                           'valid-acc': valid_acc,
#                           'score': score,}
# print(exp_name_tag)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 15, 15, 64)       